# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("weather_data.csv")
weather_df.head()


,city,date,lat,lng,temp,humd,wind,cloud
0,Hermanus,1613067774,-34.4187,19.2345,75.00,80,1.99,0
1,Cabo San Lucas,1613067906,22.8909,-109.9124,75.00,60,3.44,20
2,Lorengau,1613067997,-2.0226,147.2712,80.92,77,7.87,100
3,Yellowknife,1613068072,62.4560,-114.3525,-4.00,84,11.50,90
4,São João da Barra,1613067786,-21.6403,-41.0511,86.00,58,20.71,20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = weather_df["humd"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["lat", "lng"]]

In [6]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity,point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
fit_weather_df = weather_df.loc[(weather_df["temp"] > 70) & (weather_df["temp"] < 80) & (weather_df["cloud"] < 50), :]
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df.reset_index(inplace=True)
del fit_weather_df['index']
fit_weather_df

,city,date,lat,lng,temp,humd,wind,cloud
0,Hermanus,1613067774,-34.4187,19.2345,75.00,80,1.99,0
1,Cabo San Lucas,1613067906,22.8909,-109.9124,75.00,60,3.44,20
2,East London,1613067964,-33.0153,27.9116,75.20,88,11.50,20
3,Atuona,1613068106,-9.8000,-139.0333,78.53,75,13.53,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotels = []

for i in range(len(fit_weather_df)):
    lat = fit_weather_df.loc[i]['lat']
    lng = fit_weather_df.loc[i]['lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotels",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
fit_weather_df["hotel name"] = hotels
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df.head()


,city,date,lat,lng,temp,humd,wind,cloud,hotel name,Hotel Name
0,Hermanus,1613067774,-34.4187,19.2345,75.00,80,1.99,0,Hermanus,Hermanus
1,Cabo San Lucas,1613067906,22.8909,-109.9124,75.00,60,3.44,20,Cabo San Lucas,Cabo San Lucas
2,East London,1613067964,-33.0153,27.9116,75.20,88,11.50,20,East London,East London
3,Atuona,1613068106,-9.8000,-139.0333,78.53,75,13.53,1,Atuona,Atuona


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fit_weather_df.iterrows()]
locations = fit_weather_df[["lat", "lng"]]

KeyError: 'country'

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig
